<a href="https://colab.research.google.com/github/Chinomso1995/ESPIDA/blob/main/codigo_para_el_tesis_final_edits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Results for chile dataset

In [ ]:
import os
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def calculate_dist(X1, X2, Y1, Y2):  # Calculate Euclidean distance
    return ((X1 - X2) ** 2 + (Y1 - Y2) ** 2) ** 0.5

def create_tensor(dataset, look_back):
    dataX, dataY = [], []
    for i in range(look_back, len(dataset)):
        matrix = []
        current = dataset.iloc[i]
        for j in range(i - look_back, i):
            previous = dataset.iloc[j]
            dist = calculate_dist(current["Longitude"], previous["Longitude"],
                                  current["Latitude"], previous["Latitude"])
            dt = (current["UTC_Date"] - previous["UTC_Date"]).total_seconds() # Convert Timedelta to seconds
            dm = current["Magnitude"] - previous["Magnitude"]
            matrix.append([
                current["Latitude"], current["Longitude"], current["Depth_km"], current["Magnitude"],
                previous["Latitude"], previous["Longitude"], previous["Depth_km"], previous["Magnitude"],
                previous["Label"], dist, dt, dm
            ])
        dataX.append(matrix)
        dataY.append(current["Label"])
    return np.array(dataX), np.array(dataY)

def labeler(df, multiclass=False):
    columns = ["Latitude", "Longitude", "Depth_km", "Magnitude", "UTC_Date", "Date", "cluster"]
    data = df[columns]
    df = df.sort_values(by="cluster", ascending=True)

    clusters = list(set(data.cluster.values))
    columns.append("Label")
    df_out = pd.DataFrame(columns=columns)

    for cluster in clusters:
        filter_by_cluster = data[data["cluster"] == cluster]
        max_mag = filter_by_cluster.sort_values(by="Magnitude", ascending=False)
        max_mag_index = list(max_mag.index.values)
        time = list(max_mag.UTC_Date.values)[0]
        max_mag_index = max_mag_index[0]

        if not multiclass:
            if cluster == -1:
                filter_by_cluster["Label"] = [-1] * len(filter_by_cluster)
            else:
                label = []
                for _, row in filter_by_cluster.iterrows():
                    if row.UTC_Date <= time:
                        label.append(-1)
                    else:
                        label.append(1)
                filter_by_cluster["Label"] = label
        else:
            if cluster == -1:
                filter_by_cluster["Label"] = [1] * len(filter_by_cluster)
            else:
                label = []
                for _, row in filter_by_cluster.iterrows():
                    if row.UTC_Date < time:
                        label.append(0)  # Foreshock
                    elif row.UTC_Date == time:
                        label.append(1)  # Mainshock
                    else:
                        label.append(2)  # Aftershock
                filter_by_cluster["Label"] = label

        df_out = pd.concat([df_out, filter_by_cluster])
        df_out = df_out.sort_values(by="UTC_Date")

    df = df_out[["Latitude", "Longitude", "Depth_km", "Magnitude", "Date", "UTC_Date", "cluster", "Label"]]
    return df

In [ ]:
def from_dataframe_to_tensor(df_train,df_test,df_validation,window):

    #Create Tensor
    trainX,trainY=create_tensor(df_train,window)
    testX,testY=create_tensor(df_test,window)
    valX,valY=create_tensor(df_validation,window)


    #Normalize the matrix of inputs
    shape_tensor_trainX=trainX.shape
    trainX=trainX.reshape(trainX.shape[0]*trainX.shape[1], 12)

    shape_tensor_valX=valX.shape
    valX=valX.reshape(valX.shape[0]*valX.shape[1], 12)

    shape_tensor_testX=testX.shape
    testX=testX.reshape(testX.shape[0]*testX.shape[1], 12)

    #Aplly the same normalization rules to the train,val and test data
    scaler = preprocessing.StandardScaler().fit(trainX)
    trainX=scaler.transform(trainX)

    testX=scaler.transform(testX)
    valX=scaler.transform(valX)

    #Re-reshape to the tensor 3D shape
    trainX=trainX.reshape(shape_tensor_trainX)
    valX=valX.reshape(shape_tensor_valX)
    testX=testX.reshape(shape_tensor_testX)

    print("Normalization process successfully completed")
    input_shape = trainX.shape[1:]
    return trainX,trainY,testX,testY,valX,valY

In [ ]:
# Load and label your dataset
df = pd.read_csv("Clustered_EarthquakesChile_STDBSCANEV.csv")
df["UTC_Date"] = pd.to_datetime(df["UTC_Date"], errors='coerce')

# Use multiclass labeling
df = labeler(df, multiclass=True)
df.to_csv("chile_earthquake_data_updated.csv", index=False)

# Split by year ranges
df_train = df[(df["UTC_Date"].dt.year >= 2000) & (df["UTC_Date"].dt.year <= 2022)]
df_validation = df[df["UTC_Date"].dt.year == 2023]
df_test = df[df["UTC_Date"].dt.year == 2024]

# Sort and drop date column for model input
df_train = df_train.sort_values(by="UTC_Date").drop(columns=["Date"])
df_train.to_csv("chile_earthquake_data_train.csv", index=False)
df_validation = df_validation.sort_values(by="UTC_Date").drop(columns=["Date"])
df_validation.to_csv("chile_earthquake_data_validation.csv", index=False)
df_test = df_test.sort_values(by="UTC_Date").drop(columns=["Date"])
df_test.to_csv("chile_earthquake_data_test.csv", index=False)

# Done!
print("Train:", df_train.shape)
print("Validation:", df_validation.shape)
print("Test:", df_test.shape)


Train: (104086, 7)
Validation: (5616, 7)
Test: (918, 7)


**LSTM** / TRANSFORMER

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras

# Define evaluation function
def evaluate_model(model, testX, df_test, window, is_multiclass, model_type):
    yhat = model.predict(testX, verbose=0)

    if not is_multiclass:
        y_pred = np.where(yhat > 0.5, 1, -1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['No replica', 'Replica']
        labels = [-1, 1]
    else:
        y_pred = np.argmax(yhat, axis=1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['Precursor', 'Principal', 'Replica']
        labels = [0, 1, 2]

    print(f"\n🧾 {model_type} Classification Report (Window={window})")
    print(classification_report(y_true, y_pred, target_names=target_names))
    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred, labels=labels))

    # Save report
    with open(f"{model_type.lower()}_report_window_{window}.txt", "w") as f:
        f.write(f"{model_type} Classification Report (Window={window})\n")
        f.write(classification_report(y_true, y_pred, target_names=target_names))
        f.write("\nConfusion Matrix\n")
        f.write(str(confusion_matrix(y_true, y_pred, labels=labels)))

# === Configuration ===
window_sizes = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
is_multiclass = True  # set to False if binary classification

# === Loop over window sizes ===
for window in window_sizes:
    print(f"\n🚀 Processing window size: {window}")

    # Step 1: Recreate tensors
    trainX, trainY, testX, testY, valX, valY = from_dataframe_to_tensor(df_train, df_test, df_validation, window)

    # Step 2: Compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(trainY), y=trainY)
    class_weight_dict = dict(enumerate(class_weights))

    # Step 3: Train LSTM model
    lstm_model = Sequential([
        LSTM(64, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(trainY)), activation='softmax' if is_multiclass else 'sigmoid')
    ])
    lstm_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy' if is_multiclass else 'binary_crossentropy',
        metrics=['accuracy']
    )
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lstm_model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1,
        callbacks=[early_stop]
    )

    # Save and evaluate LSTM
    lstm_model_name = f"earthquake_lstm_model_window_{window}.keras"
    lstm_model.save(lstm_model_name)
    print(f"✅ LSTM model saved as: {lstm_model_name}")
    evaluate_model(lstm_model, testX, df_test, window, is_multiclass, "LSTM")

    # Step 4: Train Transformer model
    input_layer = Input(shape=(trainX.shape[1], trainX.shape[2]))
    attention_output = MultiHeadAttention(num_heads=4, key_dim=trainX.shape[2])(input_layer, input_layer)
    attention_output = Dropout(0.3)(attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(input_layer + attention_output)
    ffn = Dense(64, activation='relu')(attention_output)
    ffn = Dropout(0.3)(ffn)
    ffn = Dense(trainX.shape[2])(ffn)
    ffn_output = LayerNormalization(epsilon=1e-6)(attention_output + ffn)
    pooled = GlobalAveragePooling1D()(ffn_output)
    output = Dense(len(np.unique(trainY)), activation='softmax' if is_multiclass else 'sigmoid')(pooled)

    transformer_model = Model(inputs=input_layer, outputs=output)
    transformer_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy' if is_multiclass else 'binary_crossentropy',
        metrics=['accuracy']
    )
    transformer_model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1,
        callbacks=[early_stop]
    )

    # Save and evaluate Transformer
    transformer_model_name = f"earthquake_transformer_model_window_{window}.keras"
    transformer_model.save(transformer_model_name)
    print(f"✅ Transformer model saved as: {transformer_model_name}")
    evaluate_model(transformer_model, testX, df_test, window, is_multiclass, "Transformer")



🚀 Processing window size: 10
Normalization process successfully completed
Epoch 1/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.6180 - loss: 0.8360 - val_accuracy: 0.6306 - val_loss: 0.7977
Epoch 2/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 19s 5ms/step - accuracy: 0.6957 - loss: 0.7279 - val_accuracy: 0.6197 - val_loss: 0.8232
Epoch 3/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.7132 - loss: 0.6994 - val_accuracy: 0.6516 - val_loss: 0.7360
Epoch 4/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.7233 - loss: 0.6698 - val_accuracy: 0.6687 - val_loss: 0.7790
Epoch 5/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 20s 5ms/step - accuracy: 0.7384 - loss: 0.6535 - val_accuracy: 0.6787 - val_loss: 0.7548
Epoch 6/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.7445 - loss: 0.6418 - val_accuracy: 0.7158 - val_loss: 0.6863
Epoch 7/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 21s 6ms/step - accuracy: 0.7572 - loss: 0.6347 - val_accuracy: 0.7046 - val_loss: 0.6577
E

**STTN**

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# === Configuration ===
window_sizes = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
is_multiclass = True  # or False for binary classification

# === Define Evaluation Function ===
def evaluate_model(model, testX, df_test, window, is_multiclass, model_type):
    yhat = model.predict(testX, verbose=0)

    if not is_multiclass:
        y_pred = np.where(yhat > 0.5, 1, -1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['No replica', 'Replica']
        labels = [-1, 1]
    else:
        y_pred = np.argmax(yhat, axis=1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['Precursor', 'Principal', 'Replica']
        labels = [0, 1, 2]

    print(f"\n🧾 {model_type} Classification Report (Window={window})")
    print(classification_report(y_true, y_pred, target_names=target_names))
    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred, labels=labels))

    with open(f"{model_type.lower()}_report_window_{window}.txt", "w") as f:
        f.write(f"{model_type} Classification Report (Window={window})\n")
        f.write(classification_report(y_true, y_pred, target_names=target_names))
        f.write("\nConfusion Matrix\n")
        f.write(str(confusion_matrix(y_true, y_pred, labels=labels)))

# === Loop Over Window Sizes ===
for window in window_sizes:
    print(f"\n🚀 STTN: Processing window size: {window}")

    # Step 1: Convert to tensors
    trainX, trainY, testX, testY, valX, valY = from_dataframe_to_tensor(df_train, df_test, df_validation, window)

    # Step 2: Compute class weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(trainY), y=trainY)
    class_weight_dict = dict(enumerate(class_weights))

    # Step 3: Define STTN Model
    input_layer = Input(shape=(trainX.shape[1], trainX.shape[2]))  # [time_steps, features]

    # Temporal Attention Block
    temp_attn = MultiHeadAttention(num_heads=4, key_dim=trainX.shape[2])(input_layer, input_layer)
    temp_attn = Dropout(0.3)(temp_attn)
    temp_out = LayerNormalization(epsilon=1e-6)(input_layer + temp_attn)

    # Feedforward Block
    ffn = Dense(64, activation='relu')(temp_out)
    ffn = Dropout(0.3)(ffn)
    ffn = Dense(trainX.shape[2])(ffn)
    ffn_out = LayerNormalization(epsilon=1e-6)(temp_out + ffn)

    # Global Pooling and Output
    pooled = GlobalAveragePooling1D()(ffn_out)
    output = Dense(len(np.unique(trainY)), activation='softmax' if is_multiclass else 'sigmoid')(pooled)

    # Compile Model
    sttn_model = Model(inputs=input_layer, outputs=output)
    sttn_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy' if is_multiclass else 'binary_crossentropy',
        metrics=['accuracy']
    )

    # Step 4: Train STTN
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    sttn_model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1,
        callbacks=[early_stop]
    )

    # Step 5: Save & Evaluate
    model_name = f"earthquake_sttn_model_window_{window}.keras"
    sttn_model.save(model_name)
    print(f"✅ STTN model saved as: {model_name}")
    evaluate_model(sttn_model, testX, df_test, window, is_multiclass, "Spatio-Temporal Transformer")



🚀 STTN: Processing window size: 10
Normalization process successfully completed
Epoch 1/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - accuracy: 0.6140 - loss: 0.8351 - val_accuracy: 0.5974 - val_loss: 0.8816
Epoch 2/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.7005 - loss: 0.7309 - val_accuracy: 0.6577 - val_loss: 0.7381
Epoch 3/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.7245 - loss: 0.6856 - val_accuracy: 0.6138 - val_loss: 0.8324
Epoch 4/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7282 - loss: 0.6680 - val_accuracy: 0.6589 - val_loss: 0.7506
Epoch 5/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7415 - loss: 0.6467 - val_accuracy: 0.6423 - val_loss: 0.7982
Epoch 6/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7495 - loss: 0.6367 - val_accuracy: 0.6384 - val_loss: 0.7958
Epoch 7/100
3253/3253 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7540 - loss: 0.6307 - val_accuracy: 0.6775 - val_loss: 0.

**Result for japan dataset**

In [ ]:
import os
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd

def calculate_dist(X1, X2, Y1, Y2):  # Calculate Euclidean distance
    return ((X1 - X2) ** 2 + (Y1 - Y2) ** 2) ** 0.5

def create_tensor(dataset, look_back):
    dataX, dataY = [], []
    for i in range(look_back, len(dataset)):
        matrix = []
        current = dataset.iloc[i]
        for j in range(i - look_back, i):
            previous = dataset.iloc[j]
            dist = calculate_dist(current["longitude"], previous["longitude"],
                                  current["latitude"], previous["latitude"])
            dt = (pd.to_datetime(current["time"]) - pd.to_datetime(previous["time"])).total_seconds()
            dm = current["mag"] - previous["mag"]
            matrix.append([
                current["latitude"], current["longitude"], current["depth"], current["mag"],
                previous["latitude"], previous["longitude"], previous["depth"], previous["mag"],
                previous["Label"], dist, dt, dm
            ])
        dataX.append(matrix)
        dataY.append(current["Label"])
    return np.array(dataX), np.array(dataY)

def labeler(df, multiclass=False):
    df["UTC_Date"] = pd.to_datetime(df["time"], utc=True, errors='coerce')

    columns = ["latitude", "longitude", "depth", "mag", "UTC_Date", "cluster"]
    data = df[columns]
    df = df.sort_values(by="cluster", ascending=True)

    clusters = list(set(data.cluster.values))
    columns.append("Label")
    df_out = pd.DataFrame(columns=columns)

    for cluster in clusters:
        filter_by_cluster = data[data["cluster"] == cluster]
        max_mag = filter_by_cluster.sort_values(by="mag", ascending=False)
        max_mag_index = list(max_mag.index.values)
        time = pd.to_datetime(max_mag.iloc[0]["UTC_Date"], utc=True)  # Ensure tz-aware
        max_mag_index = max_mag_index[0]

        if not multiclass:
            if cluster == -1:
                filter_by_cluster["Label"] = [-1] * len(filter_by_cluster)
            else:
                label = []
                for _, row in filter_by_cluster.iterrows():
                    if row.UTC_Date <= time:
                        label.append(-1)
                    else:
                        label.append(1)
                filter_by_cluster["Label"] = label
        else:
            if cluster == -1:
                filter_by_cluster["Label"] = [1] * len(filter_by_cluster)
            else:
                label = []
                for _, row in filter_by_cluster.iterrows():
                    if row.UTC_Date < time:
                        label.append(0)  # Foreshock
                    elif row.UTC_Date == time:
                        label.append(1)  # Mainshock
                    else:
                        label.append(2)  # Aftershock
                filter_by_cluster["Label"] = label

        df_out = pd.concat([df_out, filter_by_cluster])
        df_out = df_out.sort_values(by="UTC_Date")

    df = df_out[["latitude", "longitude", "depth", "mag", "UTC_Date", "cluster", "Label"]]
    print(df)
    return df





In [ ]:
def from_dataframe_to_tensor(df_train,df_test,df_validation,window):

    #Create Tensor
    trainX,trainY=create_tensor(df_train,window)
    testX,testY=create_tensor(df_test,window)
    valX,valY=create_tensor(df_validation,window)


    #Normalize the matrix of inputs
    shape_tensor_trainX=trainX.shape
    trainX=trainX.reshape(trainX.shape[0]*trainX.shape[1], 12)

    shape_tensor_valX=valX.shape
    valX=valX.reshape(valX.shape[0]*valX.shape[1], 12)

    shape_tensor_testX=testX.shape
    testX=testX.reshape(testX.shape[0]*testX.shape[1], 12)

    #Aplly the same normalization rules to the train,val and test data
    scaler = preprocessing.StandardScaler().fit(trainX)
    trainX=scaler.transform(trainX)

    testX=scaler.transform(testX)
    valX=scaler.transform(valX)

    #Re-reshape to the tensor 3D shape
    trainX=trainX.reshape(shape_tensor_trainX)
    valX=valX.reshape(shape_tensor_valX)
    testX=testX.reshape(shape_tensor_testX)

    print("Normalization process successfully completed")
    input_shape = trainX.shape[1:]
    return trainX,trainY,testX,testY,valX,valY

In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("Japan-Comparison_Clusters_STDBSCANEV.csv")





df = labeler(df, multiclass=True)
df.to_csv("japan_earthquake_data_updated.csv", index=False)

# Split based on year in UTC_Date
df_train = df[(df["UTC_Date"].dt.year >= 2001) & (df["UTC_Date"].dt.year <= 2016)]
df_train.to_csv("japan_earthquake_data_train.csv", index=False)
df_validation = df[df["UTC_Date"].dt.year == 2017]
df_validation.to_csv("japan_earthquake_data_validation.csv", index=False)
df_test = df[df["UTC_Date"].dt.year == 2018]
df_test.to_csv("japan_earthquake_data_test.csv", index=False)

# Sort and drop 'Date' if exists (only labeler() may have added it)
df_train = df_train.sort_values(by="UTC_Date").drop(columns=["Date"], errors="ignore")
df_validation = df_validation.sort_values(by="UTC_Date").drop(columns=["Date"], errors="ignore")
df_test = df_test.sort_values(by="UTC_Date").drop(columns=["Date"], errors="ignore")

# Done!
print("Train:", df_train.shape)
print("Validation:", df_validation.shape)
print("Test:", df_test.shape)

df_train = df_train.rename(columns={"UTC_Date": "time"})
df_validation = df_validation.rename(columns={"UTC_Date": "time"})
df_test = df_test.rename(columns={"UTC_Date": "time"})




       latitude  longitude  depth  mag                         UTC_Date  \
14091    37.218    138.431   33.0  4.9 2001-01-02 10:53:23.280000+00:00   
14090    32.239    141.508   33.0  4.9 2001-01-02 20:29:49.670000+00:00   
14089    45.301    149.941   33.0  4.6 2001-01-03 09:32:54.710000+00:00   
14088    43.932    147.813   33.0  5.9 2001-01-03 14:47:49.540000+00:00   
14087    36.981    138.629   21.4  5.3 2001-01-04 04:18:21.430000+00:00   
...         ...        ...    ...  ...                              ...   
13635    30.077    142.789   33.0  4.5                              NaT   
13659    31.229    140.263   33.0  4.7                              NaT   
13681    34.640    135.719  389.7  4.5                              NaT   
13708    38.179    140.695   84.4  4.7                              NaT   
13907    44.410    148.168   98.0  4.9                              NaT   

      cluster Label  
14091      -1     1  
14090      -1     1  
14089      -1     1  
14088      

**LSTM / TRANSFORMER**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras


# === Configuration ===
window_sizes = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
is_multiclass = True

# === Evaluation function ===
def evaluate_model(model, testX, df_test, window, is_multiclass, model_type):
    yhat = model.predict(testX, verbose=0)

    if not is_multiclass:
        y_pred = np.where(yhat > 0.5, 1, -1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['No replica', 'Replica']
        labels = [-1, 1]
    else:
        y_pred = np.argmax(yhat, axis=1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['Precursor', 'Principal', 'Replica']
        labels = [0, 1, 2]

    print(f"\n🧾 {model_type} Classification Report (Window={window})")
    print(classification_report(y_true, y_pred, target_names=target_names))
    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred, labels=labels))

    with open(f"{model_type.lower()}_report_window_{window}.txt", "w") as f:
        f.write(f"{model_type} Classification Report (Window={window})\n")
        f.write(classification_report(y_true, y_pred, target_names=target_names))
        f.write("\nConfusion Matrix\n")
        f.write(str(confusion_matrix(y_true, y_pred, labels=labels)))

# === Loop through window sizes ===
for window in window_sizes:
    print(f"\n🚀 Processing window size: {window}")

    # From your own function to generate trainX, trainY, etc.
    trainX, trainY, testX, testY, valX, valY = from_dataframe_to_tensor(df_train, df_test, df_validation, window)

    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(trainY), y=trainY)
    class_weight_dict = dict(enumerate(class_weights))

    # === LSTM Model ===
    lstm_model = Sequential([
        LSTM(64, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(trainY)), activation='softmax' if is_multiclass else 'sigmoid')
    ])
    lstm_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy' if is_multiclass else 'binary_crossentropy',
        metrics=['accuracy']
    )
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lstm_model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1,
        callbacks=[early_stop]
    )
    lstm_model_name = f"earthquake_lstm_model_window_{window}.keras"
    lstm_model.save(lstm_model_name)
    print(f"✅ LSTM model saved as: {lstm_model_name}")
    evaluate_model(lstm_model, testX, df_test, window, is_multiclass, "LSTM")

    # === Transformer Model ===
    input_layer = Input(shape=(trainX.shape[1], trainX.shape[2]))
    attention_output = MultiHeadAttention(num_heads=4, key_dim=trainX.shape[2])(input_layer, input_layer)
    attention_output = Dropout(0.3)(attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(input_layer + attention_output)
    ffn = Dense(64, activation='relu')(attention_output)
    ffn = Dropout(0.3)(ffn)
    ffn = Dense(trainX.shape[2])(ffn)
    ffn_output = LayerNormalization(epsilon=1e-6)(attention_output + ffn)
    pooled = GlobalAveragePooling1D()(ffn_output)
    output = Dense(len(np.unique(trainY)), activation='softmax' if is_multiclass else 'sigmoid')(pooled)

    transformer_model = Model(inputs=input_layer, outputs=output)
    transformer_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy' if is_multiclass else 'binary_crossentropy',
        metrics=['accuracy']
    )
    transformer_model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1,
        callbacks=[early_stop]
    )
    transformer_model_name = f"earthquake_transformer_model_window_{window}.keras"
    transformer_model.save(transformer_model_name)
    print(f"✅ Transformer model saved as: {transformer_model_name}")
    evaluate_model(transformer_model, testX, df_test, window, is_multiclass, "Transformer")


🚀 Processing window size: 10
Normalization process successfully completed
Epoch 1/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.6186 - loss: 0.8687 - val_accuracy: 0.9508 - val_loss: 0.2508
Epoch 2/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7825 - loss: 0.6874 - val_accuracy: 0.9547 - val_loss: 0.3102
Epoch 3/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7851 - loss: 0.6617 - val_accuracy: 0.9587 - val_loss: 0.1984
Epoch 4/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7934 - loss: 0.6374 - val_accuracy: 0.9350 - val_loss: 0.3043
Epoch 5/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8118 - loss: 0.5798 - val_accuracy: 0.9547 - val_loss: 0.2798
Epoch 6/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7869 - loss: 0.6265 - val_accuracy: 0.9528 - val_loss: 0.2339
Epoch 7/100
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8146 - loss: 0.5784 - val_accuracy: 0.9685 - val_loss: 0.1641
Epoch 8/100
403/403 ━━

**Result for mexico dataset**

In [ ]:
import os
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd

def calculate_dist(X1, X2, Y1, Y2):  # Calculate Euclidean distance
    return ((X1 - X2) ** 2 + (Y1 - Y2) ** 2) ** 0.5

def create_tensor(dataset, look_back):
    dataX, dataY = [], []
    for i in range(look_back, len(dataset)):
        matrix = []
        current = dataset.iloc[i]
        for j in range(i - look_back, i):
            previous = dataset.iloc[j]
            dist = calculate_dist(current["longitude"], previous["longitude"],
                                  current["latitude"], previous["latitude"])
            dt = (pd.to_datetime(current["date time"]) - pd.to_datetime(previous["date time"])) .total_seconds()
            dm = current["magnitude"] - previous["magnitude"]
            matrix.append([
                current["latitude"], current["longitude"], current["depth"], current["magnitude"],
                previous["latitude"], previous["longitude"], previous["depth"], previous["magnitude"],
                previous["Label"], dist, dt, dm
            ])
        dataX.append(matrix)
        dataY.append(current["Label"])
    return np.array(dataX), np.array(dataY)

def labeler(df, multiclass=False):
    df["UTC_Date"] = pd.to_datetime(df["date time"], utc=True, errors='coerce')

    columns = ["latitude", "longitude", "depth", "magnitude", "UTC_Date", "cluster"]
    data = df[columns]
    df = df.sort_values(by="cluster", ascending=True)

    clusters = list(set(data.cluster.values))
    columns.append("Label")
    df_out = pd.DataFrame(columns=columns)

    for cluster in clusters:
        filter_by_cluster = data[data["cluster"] == cluster]
        max_mag = filter_by_cluster.sort_values(by="magnitude", ascending=False)
        max_mag_index = list(max_mag.index.values)
        time = pd.to_datetime(max_mag.iloc[0]["UTC_Date"], utc=True)  # Ensure tz-aware
        max_mag_index = max_mag_index[0]

        if not multiclass:
            if cluster == -1:
                filter_by_cluster["Label"] = [-1] * len(filter_by_cluster)
            else:
                label = []
                for _, row in filter_by_cluster.iterrows():
                    if row.UTC_Date <= time:
                        label.append(-1)
                    else:
                        label.append(1)
                filter_by_cluster["Label"] = label
        else:
            if cluster == -1:
                filter_by_cluster["Label"] = [1] * len(filter_by_cluster)
            else:
                label = []
                for _, row in filter_by_cluster.iterrows():
                    if row.UTC_Date < time:
                        label.append(0)  # Foreshock
                    elif row.UTC_Date == time:
                        label.append(1)  # Mainshock
                    else:
                        label.append(2)  # Aftershock
                filter_by_cluster["Label"] = label

        df_out = pd.concat([df_out, filter_by_cluster])
        df_out = df_out.sort_values(by="UTC_Date")

    df = df_out[["latitude", "longitude", "depth", "magnitude", "UTC_Date", "cluster", "Label"]]
    print(df)
    return df


In [ ]:
def from_dataframe_to_tensor(df_train,df_test,df_validation,window):

    #Create Tensor
    trainX,trainY=create_tensor(df_train,window)
    testX,testY=create_tensor(df_test,window)
    valX,valY=create_tensor(df_validation,window)


    #Normalize the matrix of inputs
    shape_tensor_trainX=trainX.shape
    trainX=trainX.reshape(trainX.shape[0]*trainX.shape[1], 12)

    shape_tensor_valX=valX.shape
    valX=valX.reshape(valX.shape[0]*valX.shape[1], 12)

    shape_tensor_testX=testX.shape
    testX=testX.reshape(testX.shape[0]*testX.shape[1], 12)

    #Aplly the same normalization rules to the train,val and test data
    scaler = preprocessing.StandardScaler().fit(trainX)
    trainX=scaler.transform(trainX)

    testX=scaler.transform(testX)
    valX=scaler.transform(valX)

    #Re-reshape to the tensor 3D shape
    trainX=trainX.reshape(shape_tensor_trainX)
    valX=valX.reshape(shape_tensor_valX)
    testX=testX.reshape(shape_tensor_testX)

    print("Normalization process successfully completed")
    input_shape = trainX.shape[1:]
    return trainX,trainY,testX,testY,valX,valY

In [ ]:
import pandas as pd

# Load dataset
df_mexico = pd.read_csv("/content/Mexico-Comparison_Clusters_STDBSCANEV.csv")


df_mexico_labeled = labeler(df_mexico, multiclass=True)
df_mexico_labeled.to_csv("mexico_earthquake_data_updated.csv", index=False)

# Perform the year-based split: train 1985–2020, validation 2021, test 2022
df_train = df_mexico_labeled[(df_mexico_labeled["UTC_Date"].dt.year >= 1985) & (df_mexico_labeled["UTC_Date"].dt.year <= 2020)]
df_train.to_csv("mexico_earthquake_data_train.csv", index=False)
df_validation = df_mexico_labeled[df_mexico_labeled["UTC_Date"].dt.year == 2021]
df_validation.to_csv("mexico_earthquake_data_validation.csv", index=False)
df_test = df_mexico_labeled[df_mexico_labeled["UTC_Date"].dt.year == 2022]
df_test.to_csv("mexico_earthquake_data_test.csv", index=False)

# Sort and drop any unnecessary columns (like Date if present)
df_train = df_train.sort_values(by="UTC_Date")
df_validation = df_validation.sort_values(by="UTC_Date")
df_test = df_test.sort_values(by="UTC_Date")

# Rename 'UTC_Date' to 'time' for downstream compatibility
df_train = df_train.rename(columns={"UTC_Date": "time"})
df_validation = df_validation.rename(columns={"UTC_Date": "time"})
df_test = df_test.rename(columns={"UTC_Date": "time"})

# Display dataset shapes
df_train.shape, df_validation.shape, df_test.shape

       latitude  longitude  depth  magnitude                  UTC_Date  \
0       14.3600   -93.1700   33.0        4.7 1985-01-05 07:30:00+00:00   
1       16.3600   -99.0100   33.0        4.6 1985-01-05 13:51:00+00:00   
2       13.6900   -92.8300   33.0        4.4 1985-01-05 18:09:00+00:00   
3       13.5800   -92.7600  100.0        4.4 1985-01-05 18:18:00+00:00   
4       16.7900   -99.6200   33.0        4.7 1985-01-07 09:08:00+00:00   
...         ...        ...    ...        ...                       ...   
40557   18.0925  -103.2250   11.9        4.0 2022-12-31 07:15:00+00:00   
40556   15.2958   -93.2825   76.4        4.9 2022-12-31 09:38:00+00:00   
40558   17.1185   -94.9105  123.5        4.0 2022-12-31 10:57:00+00:00   
40559   15.2108   -93.3647   73.1        4.0 2022-12-31 12:36:00+00:00   
40560   16.3808  -100.6120   15.9        4.0 2022-12-31 15:50:00+00:00   

      cluster Label  
0          -1     1  
1          -1     1  
2          -1     1  
3          -1     1  
4

((35463, 7), (2628, 7), (2470, 7))

**LSTM / TRANSFORMER**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras


df_train = df_train.rename(columns={"time": "date time"})
df_validation = df_validation.rename(columns={"time": "date time"})
df_test = df_test.rename(columns={"time": "date time"})

# === Configuration ===
window_sizes = [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
is_multiclass = True

# === Evaluation function ===
def evaluate_model(model, testX, df_test, window, is_multiclass, model_type):
    yhat = model.predict(testX, verbose=0)

    if not is_multiclass:
        y_pred = np.where(yhat > 0.5, 1, -1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['No replica', 'Replica']
        labels = [-1, 1]
    else:
        y_pred = np.argmax(yhat, axis=1)
        y_true = list(df_test.iloc[window:]["Label"].values)
        target_names = ['Precursor', 'Principal', 'Replica']
        labels = [0, 1, 2]

    print(f"\n🧾 {model_type} Classification Report (Window={window})")
    print(classification_report(y_true, y_pred, target_names=target_names))
    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred, labels=labels))

    with open(f"{model_type.lower()}_report_window_{window}.txt", "w") as f:
        f.write(f"{model_type} Classification Report (Window={window})\n")
        f.write(classification_report(y_true, y_pred, target_names=target_names))
        f.write("\nConfusion Matrix\n")
        f.write(str(confusion_matrix(y_true, y_pred, labels=labels)))

# === Loop through window sizes ===
for window in window_sizes:
    print(f"\n🚀 Processing window size: {window}")

    # From your own function to generate trainX, trainY, etc.
    trainX, trainY, testX, testY, valX, valY = from_dataframe_to_tensor(df_train, df_test, df_validation, window)

    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(trainY), y=trainY)
    class_weight_dict = dict(enumerate(class_weights))

    # === LSTM Model ===
    lstm_model = Sequential([
        LSTM(64, input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=False),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(len(np.unique(trainY)), activation='softmax' if is_multiclass else 'sigmoid')
    ])
    lstm_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy' if is_multiclass else 'binary_crossentropy',
        metrics=['accuracy']
    )
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lstm_model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1,
        callbacks=[early_stop]
    )
    lstm_model_name = f"earthquake_lstm_mexico_model_window_{window}.keras"
    lstm_model.save(lstm_model_name)
    print(f"✅ LSTM model saved as: {lstm_model_name}")
    evaluate_model(lstm_model, testX, df_test, window, is_multiclass, "LSTM")

    # === Transformer Model ===
    input_layer = Input(shape=(trainX.shape[1], trainX.shape[2]))
    attention_output = MultiHeadAttention(num_heads=4, key_dim=trainX.shape[2])(input_layer, input_layer)
    attention_output = Dropout(0.3)(attention_output)
    attention_output = LayerNormalization(epsilon=1e-6)(input_layer + attention_output)
    ffn = Dense(64, activation='relu')(attention_output)
    ffn = Dropout(0.3)(ffn)
    ffn = Dense(trainX.shape[2])(ffn)
    ffn_output = LayerNormalization(epsilon=1e-6)(attention_output + ffn)
    pooled = GlobalAveragePooling1D()(ffn_output)
    output = Dense(len(np.unique(trainY)), activation='softmax' if is_multiclass else 'sigmoid')(pooled)

    transformer_model = Model(inputs=input_layer, outputs=output)
    transformer_model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy' if is_multiclass else 'binary_crossentropy',
        metrics=['accuracy']
    )
    transformer_model.fit(
        trainX, trainY,
        validation_data=(valX, valY),
        epochs=100,
        batch_size=32,
        class_weight=class_weight_dict,
        verbose=1,
        callbacks=[early_stop]
    )
    transformer_model_name = f"earthquake_transformer_mexico_model_window_{window}.keras"
    transformer_model.save(transformer_model_name)
    print(f"✅ Transformer model saved as: {transformer_model_name}")
    evaluate_model(transformer_model, testX, df_test, window, is_multiclass, "Transformer")


🚀 Processing window size: 10
Normalization process successfully completed
Epoch 1/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.6231 - loss: 0.8124 - val_accuracy: 0.5680 - val_loss: 0.8659
Epoch 2/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.7571 - loss: 0.6506 - val_accuracy: 0.4687 - val_loss: 1.0222
Epoch 3/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.7755 - loss: 0.6237 - val_accuracy: 0.5642 - val_loss: 0.8916
Epoch 4/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.7758 - loss: 0.6311 - val_accuracy: 0.6654 - val_loss: 0.7273
Epoch 5/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.7976 - loss: 0.5862 - val_accuracy: 0.6184 - val_loss: 0.8436
Epoch 6/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.7796 - loss: 0.5875 - val_accuracy: 0.6662 - val_loss: 0.7735
Epoch 7/100
1108/1108 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.8006 - loss: 0.5764 - val_accuracy: 0.6058 - val_loss: 0.